In [15]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# --------------- Import packages --------------
import pandas as pd
import geopandas as gpd
from shapely import Point
import folium
import re
import numpy as np
%config InlineBackend.figure_format ='retina'

In [2]:
# --------------- Entering US county data ------------------
# Read in US counties shapefiles
us_counties = gpd.read_file('~/Human Trafficking Project/Data/usShapeFiles/cb_2018_us_county_500k.shp')

# Combine the county name and state FIPS to give each county a unique name
us_counties['name_and_fips'] = us_counties[["NAME","STATEFP"]].agg(" ".join, axis=1)

In [ ]:
# ---------------- Counting US airports by county -----------------
# Read in csv containing all U.S. airports
us_airports = pd.read_csv('~/Human Trafficking Project/Data/us-airports.csv')

# Limit the dataset to only medium and large airports.
us_airports['airport'] = us_airports['type'].isin(['large_airport', 'medium_airport'])
us_airports_v2 = us_airports[us_airports['airport']]

# Create a geometry column out to the longitude and latitude column
us_airports_v2['coord'] = us_airports_v2.apply(lambda row: Point(row['longitude_deg'], row['latitude_deg']), axis=1)

# Create a GeoDataFrame using the geometry column
us_airports_gdf = gpd.GeoDataFrame(us_airports_v2, geometry=us_airports_v2['coord'], crs='EPSG:4326')

#a function to count the number of points within polygons in a GeoDataFrame
def count_points_in_polygons(points_gdf, polygons_gdf):
    """
    Count how many points fall into each polygon in polygons_gdf.

    Parameters:
    - points_gdf: GeoDataFrame containing points.
    - polygons_gdf: GeoDataFrame containing polygons.

    Returns:
    - DataFrame with polygon IDs and corresponding point counts.
    """
    # Ensure points and polygons have the same CRS
    points_gdf = points_gdf.to_crs(polygons_gdf.crs)

    # Perform spatial join to count points within each polygon
    points_in_polygons = gpd.sjoin(points_gdf, polygons_gdf, op='within')

    # Group by polygon and count points
    point_counts = points_in_polygons.groupby('name_and_fips').size().reset_index(name='point_count')

    return point_counts

#Counting the number of airports per county
airport_count = count_points_in_polygons(us_airports_gdf, us_counties)

# Merging the airport counts back onto the counties GeoDataFrame
airport_counts= pd.merge(airport_count, us_counties, left_on="name_and_fips", right_on="name_and_fips", how='left')
airport_count_gdf = gpd.GeoDataFrame(airport_counts, geometry='geometry', crs='EPSG:4326') 

# Create a list of states and territory FIPS to remove for mapping
exclude_statefps = ['02', '15', '66', '72', '60','61','62','63', '64', '66','68', '69', '70', '74','78', '81','84','86','67','89','71','76','95','79']

# Filtering the DataFrame
lower_48 = us_counties[~us_counties['STATEFP'].isin(exclude_statefps)]

# Plot the GeoDataFrame after

In [ ]:
# ------------- Counting major roadways by US county -------------
# Read in Census shapefile for major US roads
roads = gpd.read_file('~/Human Trafficking Project/Data/USRoads/tl_2021_us_primaryroads.shp')

# Limit the dataset to only those that are interstate
roads['interstate']=roads['RTTYP'].isin(['I'])
roadsv2 = roads[roads['interstate']]

# Define a fucntion to count linestrings in a polygon
def count_linestrings_in_polygons(linestrings_gdf, polygons_gdf):
    """
    Count how many linestrings fall into each polygon in polygons_gdf.

    Parameters:
    - linestrings_gdf: GeoDataFrame containing linestrings.
    - polygons_gdf: GeoDataFrame containing polygons.

    Returns:
    - DataFrame with polygon IDs and corresponding linestring counts.
    """
    # Ensure linestrings and polygons have the same CRS
    linestrings_gdf = linestrings_gdf.to_crs(polygons_gdf.crs)
    
    # Perform spatial join to find linestrings within each polygon
    linestrings_in_polygons = gpd.sjoin(linestrings_gdf, polygons_gdf, op='intersects')
    
    # Group by polygon and count linestrings
    linestring_counts = linestrings_in_polygons.groupby('name_and_fips').size().reset_index(name='linestring_count')

    return linestring_counts

#Counting the number of roadways per county 
highway_count = count_linestrings_in_polygons(roadsv2, us_counties)

# Merge the highway counts by county onto the continental dataframe to match
highway_counts= pd.merge(highway_count, lower_48, left_on="name_and_fips", right_on="name_and_fips", how='left')

# Convert the DataFrame to a GeoDataFrame
highway_count_gdf = gpd.GeoDataFrame(highway_counts, geometry='geometry', crs='EPSG:4326') 

#ax = lower_48.plot(color='grey')
#highway_count_gdf.plot(column='linestring_count', ax=ax, figsize=(50,5))

# # Export the highway count geodataframe as a csv
# highway_count_gdf.to_csv('highway_count.csv', index=False)

In [ ]:
# ------------ Finding percentage reservation land by county -------------
# Read in Census shapfile of all reservation land in the US
reservations = gpd.read_file('~/Human Trafficking Project/Data/Reservationsv2/tl_2018_us_aiannh.shp')

# Check the CRS of the shapefile
reservations.crs

# Ensure both GeoDataFrames have the same CRS if not already
us_counties = us_counties.to_crs('EPSG:4326')  # WGS84
reservations = reservations.to_crs('EPSG:4326')  # WGS84

# ---------- Creating a folium map to confirm the accuracy of the shapefiles
# Create a Folium map centered around the mean of all coordinates
map_center = [us_counties['geometry'].centroid.y.mean(), us_counties['geometry'].centroid.x.mean()]
mymap = folium.Map(location=map_center, zoom_start=4)

# Add US counties GeoDataFrame as a GeoJSON layer to the map
folium.GeoJson(
    us_counties,
    name='US Counties',
    tooltip=folium.GeoJsonTooltip(fields=['NAME']),
).add_to(mymap)

# Add reservations GeoDataFrame as a GeoJSON layer to the map
folium.GeoJson(
    reservations,
    name='Reservations',
    style_function=lambda feature: {
        'fillColor': 'black',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(fields=['NAME']),
).add_to(mymap)

# Add Layer Control to the map (allows switching layers on/off)
folium.LayerControl().add_to(mymap)

# Display the map
#mymap


# TODO some of the code got deleted in this section, which corrected the crs of both dataframes and accurately found the overlay.
# As it wasn't critical to our analysis, I didn't finnish the work to rewrite the code. The code will run, but areas are incorrect.
# us_counties = us_counties.to_crs('EPSG:3857')  # WGS84
reservations.to_crs('EPSG:3857')  # WGS84
us_counties.to_crs('EPSG:3857')

# Overlay the reservations on top of the counties, keeping only the intersections
res_intersection = reservations.overlay(us_counties, how='intersection')

# Find the area of the resulting intersection polygons
res_intersection['area_sqm']=res_intersection.geometry.area

# Filtering the DataFrame to only the lower 48
res_intersectionv2 = res_intersection[~res_intersection['STATEFP'].isin(exclude_statefps)]

# Ensure GeoDataFrame is in a projected CRS for accurate area calculation (e.g., EPSG:3857)
us_counties_3857 = us_counties.to_crs('EPSG:3857')

# Calculate area of each polygon in square meters
us_counties_3857['county_area_sqm'] = us_counties_3857.geometry.area

overlaps = pd.merge(us_counties_3857, res_intersection, left_on="name_and_fips", right_on="name_and_fips", how='left')

overlaps['percent_res'] = (overlaps['area_sqm'] / overlaps['county_area_sqm']) * 100

overlaps.tail(30)

# Remove rows with NaN in the 'percent_res' column
overlaps.dropna(subset=['percent_res'], inplace=True)

overlaps_48=overlaps[~overlaps['name_and_fips'].isin(exclude_statefps)]

# ax=lower_48.plot(color='grey')
# overlaps_48.plot(ax=ax, column='percent_res')

In [ ]:
# --------- Beginning an analysis of human trafficking and percentage reservation ------------
# Read in the cleaned human trafficking data
htdata = pd.read_csv('~/Human Trafficking Project/Products/CleanedNIBRS2022v2.csv')

# Drop counties that don't contain human trafficking incidents
htdata.dropna(subset=['count'], inplace = True)

# Convert FIPS codes and state names to strings, then aggregating them to create unique names for each county
htdata['STATEFP'] = htdata['STATEFP'].astype(str)
htdata['NAME'] = htdata['NAME'].astype(str)
htdata['name_and_fips'] = htdata[["NAME","STATEFP"]].agg(" ".join, axis=1)

# Drop the old index column
htdata.drop(columns=['Unnamed: 0.1'], inplace=True)

# Merge the reservation/county overlap data with the human trafficking data
reshtdata= pd.merge(overlaps, htdata, left_on="name_and_fips", right_on="name_and_fips", how='left')
reshtdata.dropna(subset=['count'], inplace = True)